In [1]:
# pip install bs4 biopython

rcsb_cif_gz_dir = '/scratch/xg590/source/rcsb_cif_gz'
rcsb_cif_dir    = '/scratch/xg590/source/rcsb_cif'
cpd_cif         = '/scratch/xg590/source/cpd_cif'
adduct_bin      = '/scratch/xg590/scratch/adduct_bin'
pdb_cov_csv     = 'data/pdb_cov_20220118.csv'
doi_pkl         = 'data/doi.pkl' 

import os, pathlib, subprocess, multiprocessing 
_ = [pathlib.Path(i).mkdir(parents=True, exist_ok=True) 
        for i in [rcsb_cif_gz_dir, rcsb_cif_dir, cpd_cif, adduct_bin]] 
from IPython.display import display, HTML, SVG, clear_output
display(HTML("<style>.container { width:100% !important; }</style>")) 
import io, urllib, Bio.PDB, sys, pickle, math, json, re#, bs4
from rdkit import Chem
from rdkit.Chem import AllChem, PandasTools
PandasTools.RenderImagesInAllDataFrames(images=True)
from rdkit.Chem.Draw import IPythonConsole  
from rdkit.Geometry.rdGeometry import Point3D

class PDB_Cov_Exception(Exception):
    def __init__(self, msg=None):
        Exception.__init__(self, msg) 

In [2]:
## Mirror The DB 
def mirror_db():
    subprocess.run(f"rsync -rlpt -z --delete --port=33444 rsync.rcsb.org::ftp_data/structures/divided/mmCIF/ {rcsb_cif_gz_dir}", shell=True)
    
    from urllib.request import urlopen
    # we need to delete obsolete pdb structure before sync
    obsolete = 'ftp://ftp.wwpdb.org/pub/pdb/data/status/obsolete.dat'
    with urlopen(obsolete) as fr:
        _ = fr.readline()
        while True:
            try:
                line = fr.readline().decode()
                if not line: break
                PDBID = line.split()[2]
                pdbid = PDBID.lower()  
                os.remove(f'{rcsb_cif_dir}/{pdbid[1:3]}/{pdbid}.cif') 
                print('{} is now obselete and deleted successfully.'.format(PDBID))
            except Exception as e:
                print(e) 
                
#mirror_db()

In [3]:
## Decompress
def unzipper(pdb_id):
    cif_subdir = f'{rcsb_cif_dir}/{pdb_id[1:3]}'
    pathlib.Path(cif_subdir).mkdir(parents=True, exist_ok=True)
    cif        = f'{rcsb_cif_dir}/{pdb_id[1:3]}/{pdb_id}.cif'
    if os.path.isfile(cif): return None
    gz         = f'{rcsb_cif_gz_dir}/{pdb_id[1:3]}/{pdb_id}.cif.gz'
    proc = subprocess.run(args=['gzip', '-dc', gz], stdout=subprocess.PIPE) # unzip the zipped 
    with open(cif, 'wb') as fw: fw.write(proc.stdout)   

def decompress():
    pdb_ids = []
    for subdir in os.scandir(rcsb_cif_gz_dir): 
        if subdir.is_dir():
            for gz in os.scandir(subdir.path):
                if gz.is_file():
                    pdb_ids.append(gz.name[:4])
    
    with multiprocessing.Pool(multiprocessing.cpu_count()) as p: # unzipping in 3 cores  
        p.map(unzipper, pdb_ids, multiprocessing.cpu_count()) 
        
#decompress()

In [4]:
def get_covalent_bond_record(pdb_id, debug=False):
    try:
        pdb_id = pdb_id.lower()
        blc = Bio.PDB.MMCIF2Dict.MMCIF2Dict(f'{rcsb_cif_dir}/{pdb_id[1:3]}/{pdb_id}.cif')
        if debug: print(f'We have local copy of {pdb_id}.cif')
    except FileNotFoundError: 
        r = urllib.request.urlopen(f'http://files.rcsb.org/download/{pdb_id}.cif')
        f = io.StringIO(r.read().decode())
        if debug: print(f'We fetched {pdb_id}.cif from RCSB PDB')
        blc = Bio.PDB.MMCIF2Dict.MMCIF2Dict(f)   
    if "_struct_conn.id" not in blc: return [] # struct_conn_IS_NOT_FOUND 

    # We are not considering terminal amine although it could be the target of covalent binder. 
    Acceptable_Nucleophile = [('ASP', 'OD1'), ('ASP', 'OD2'), ('CYS', 'SG'), ('GLU', 'OE1'), ('GLU', 'OE2'), ('HIS', 'ND1'), ('HIS', 'NE2'), ('LYS', 'NZ'), ('MET', 'SD'),  ('THR', 'OG1'), ('SER', 'OG'), ('TYR', 'OH')]
    # some components are not binders (metal-containing ligand? solvent? ion?)
    Invalid_Binder = {'ALA': 'is an amino-acid residue', 'ARG': 'is an amino-acid residue', 
                      'ASN': 'is an amino-acid residue', 'ASP': 'is an amino-acid residue', 
                      'CYS': 'is an amino-acid residue', 'GLU': 'is an amino-acid residue', 
                      'GLN': 'is an amino-acid residue', 'GLY': 'is an amino-acid residue', 
                      'HIS': 'is an amino-acid residue', 'ILE': 'is an amino-acid residue', 
                      'LEU': 'is an amino-acid residue', 'LYS': 'is an amino-acid residue', 
                      'MET': 'is an amino-acid residue', 'PHE': 'is an amino-acid residue', 
                      'PRO': 'is an amino-acid residue', 'SER': 'is an amino-acid residue', 
                      'THR': 'is an amino-acid residue', 'TRP': 'is an amino-acid residue', 
                      'TYR': 'is an amino-acid residue', 'VAL': 'is an amino-acid residue', 
                      'DA': 'is a nucleotide residue', 'DC': 'is a nucleotide residue', 
                      'DG': 'is a nucleotide residue', 'DT': 'is a nucleotide residue', 
                      'DI': 'is a nucleotide residue', 'A': 'is a nucleotide residue', 
                      'C': 'is a nucleotide residue', 'G': 'is a nucleotide residue', 
                      'U': 'is a nucleotide residue', 'UNL': 'Unknown Ligand/Atom', 
                      'UNX': 'Unknown Ligand/Atom', 'UNK': 'Unknown Ligand/Atom', 'CO3': 'Anion', 
                      'SO2': 'Anion', 'SO3': 'Anion', 'SO4': 'Anion', 'NO3': 'Anion',
                      'PO3': 'Anion', 'PO4': 'Anion', 'MAN': 'Alpha-D-Mannose', 
                      'GOL': 'Pentaethylene Glycol', '1PE': 'Glycol', 'NAP': 'NADP+',
                      'BR': 'Element is not ligand ', 'HOH': 'Water or Hydroperoxide',
                      'CO2': 'Carbon Dioxide   see https://pubs.acs.org/doi/10.1021/bi960424z', 
                      'PEG': 'Diethylene Glycol', 'PAM': 'PTM: Palmitoylation', 
                      'DPN': 'AA: D-Phenylalanine', 'PYE': 'Tetrahydropyran ', 
                      'BEN': 'A non-covalent inhibitor to trypsin and Xa factor, is often used as a ligand in protein crystallography to prevent proteases from degrading a protein of interest', 
                      'GDP': "Guanosine-5'-Diphosphate", 
                      'AMP': 'Adenosine Monophosphate', 'ADP': "Adenosine-5'-Diphosphate", 
                      'ATP': "Adenosine-5'-Triphosphate", 
                      'GTP': "Guanosine-5'-Triphosphate", 'NAG': 'GlcNAc PTM', 'O': 'Oxygen ??? 1ADL',
                      'IOD': 'Iodine', 'Z': 'DNA linking', 'CL': 'Chlorine', 'AYE': 'part of UbPA', 
                      'MLY': 'methylated LYS / PTM', 'MTN': 'MTSL is an organosulfur compound that is used as a nitroxide spin label.', 
                      'PEB': 'PHYCOERYTHROBILIN', 'PUB': 'PHYCOUROBILIN', 'PVN': 'PHYCOVIOLOBILIN',
                      'VRB': 'Phycoviolobilin', 'CYC': 'PHYCOCYANOBILIN', 'HEM':'Porphyrin-Fe', 
                      'COA': 'Cofactor CoA', 'BLA': 'BILIVERDINE IX ALPHA', 'DBV': '15,16-DIHYDROBILIVERDIN', 
                      'PCA': 'Modified Residues', 'ABA': 'Modified Residues', 'CME': 'Modified Residues ',
                      'FAD': 'flavin adenine dinucleotide', 'PLP': 'Pyridoxal phosphate cofactors', 
                      'PMP': "pyridoxamine 5'-phosphate cofactors", 
                      'BME': '2-Mercaptoethanol Reducing Agents. If your protein contains cysteine residues, oxidation could become a problem and cause protein aggregation. To prevent this, keep a reducing agent such as DTT, TCEP, or 2-mercaptoethanol in your buffer.',
                      
                      'ZMP':'Phospholipids'}
    Acceptable_Element = set(['C', 'H', 'D', 'O', 'N', 'P', 'S', 'F', 'CL', 'BR', 'I', 'B'])
    cbr = []
    for i, conn_type_id in enumerate(blc["_struct_conn.conn_type_id"]):
        if conn_type_id == 'covale':
            col = []
            for j, label_entity_id in enumerate(blc['_atom_site.label_entity_id']):
                if  blc["_atom_site.label_asym_id"    ][j] == blc["_struct_conn.ptnr1_label_asym_id"     ][i] and \
                    blc["_atom_site.auth_asym_id"     ][j] == blc["_struct_conn.ptnr1_auth_asym_id"      ][i] and \
                    blc["_atom_site.label_comp_id"    ][j] == blc["_struct_conn.ptnr1_label_comp_id"     ][i] and \
                    blc["_atom_site.auth_comp_id"     ][j] == blc["_struct_conn.ptnr1_auth_comp_id"      ][i] and \
                    blc["_atom_site.label_seq_id"     ][j] == blc["_struct_conn.ptnr1_label_seq_id"      ][i] and \
                    blc["_atom_site.auth_seq_id"      ][j] == blc["_struct_conn.ptnr1_auth_seq_id"       ][i] and \
                    blc["_atom_site.pdbx_PDB_ins_code"][j] == blc["_struct_conn.pdbx_ptnr1_PDB_ins_code" ][i]:
                    for k, id_ in enumerate(blc["_entity.id"]):
                        if label_entity_id == id_:
                            col.append(blc["_entry.id"][0])
                            col.append(blc["_struct_conn.ptnr1_label_asym_id"     ][i])
                            col.append(blc["_struct_conn.ptnr1_auth_asym_id"      ][i])
                            col.append(blc["_struct_conn.ptnr1_label_comp_id"     ][i])
                            col.append(blc["_struct_conn.ptnr1_auth_comp_id"      ][i])
                            col.append(blc["_struct_conn.ptnr1_label_seq_id"      ][i])
                            col.append(blc["_struct_conn.ptnr1_auth_seq_id"       ][i])
                            col.append(blc["_struct_conn.pdbx_ptnr1_PDB_ins_code" ][i])
                            col.append(blc["_struct_conn.ptnr1_label_atom_id"     ][i])
                            col.append(blc["_struct_conn.pdbx_ptnr1_label_alt_id" ][i]) 
                            col.append(blc["_entity.type"][k])    
                            break
                    break
            for j, label_entity_id in enumerate(blc['_atom_site.label_entity_id']):
                if  blc["_atom_site.label_asym_id"    ][j] == blc["_struct_conn.ptnr2_label_asym_id"     ][i] and \
                    blc["_atom_site.auth_asym_id"     ][j] == blc["_struct_conn.ptnr2_auth_asym_id"      ][i] and \
                    blc["_atom_site.label_comp_id"    ][j] == blc["_struct_conn.ptnr2_label_comp_id"     ][i] and \
                    blc["_atom_site.auth_comp_id"     ][j] == blc["_struct_conn.ptnr2_auth_comp_id"      ][i] and \
                    blc["_atom_site.label_seq_id"     ][j] == blc["_struct_conn.ptnr2_label_seq_id"      ][i] and \
                    blc["_atom_site.auth_seq_id"      ][j] == blc["_struct_conn.ptnr2_auth_seq_id"       ][i] and \
                    blc["_atom_site.pdbx_PDB_ins_code"][j] == blc["_struct_conn.pdbx_ptnr2_PDB_ins_code" ][i]:
                    for k, id_ in enumerate(blc["_entity.id"]):
                        if label_entity_id == id_: 
                            col.append(blc["_struct_conn.ptnr2_label_asym_id"     ][i])
                            col.append(blc["_struct_conn.ptnr2_auth_asym_id"      ][i])
                            col.append(blc["_struct_conn.ptnr2_label_comp_id"     ][i])
                            col.append(blc["_struct_conn.ptnr2_auth_comp_id"      ][i])
                            col.append(blc["_struct_conn.ptnr2_label_seq_id"      ][i])
                            col.append(blc["_struct_conn.ptnr2_auth_seq_id"       ][i])
                            col.append(blc["_struct_conn.pdbx_ptnr2_PDB_ins_code" ][i])
                            col.append(blc["_struct_conn.ptnr2_label_atom_id"     ][i])
                            col.append(blc["_struct_conn.pdbx_ptnr2_label_alt_id" ][i]) 
                            col.append(blc["_entity.type"][k]) 
                            break 
                    break
            #    0 1 2   3   4   5   6 7 8 9      10 1 2   3   4   5   6 7 8 9      20    
            # 2XAZ,A,A,LEU,LEU,729,729,?,C,?,polymer,A,A,NIY,NIY,730,730,?,N,?,polymer
            if   (col[4],  col[8] ) in Acceptable_Nucleophile and col[10] == 'polymer': 
                pass
            elif (col[14], col[18]) in Acceptable_Nucleophile and col[20] == 'polymer': 
                col = [col[0]] + col[11:] + col[1:11] # Reconstruct the list
            else: 
                if debug: print(f"Invalid covalent bond record: {','.join(col)}")
                continue # skip invalid covalent modification
            if col[14] in Invalid_Binder: 
                if debug: print(f'The binder {col[14]} is invalid because of {Invalid_Binder[col[14]]}')
                continue
            binder_element_collector = [] 
            for i, type_symbol in enumerate(blc['_atom_site.type_symbol']):  
                if blc["_atom_site.auth_asym_id"][i]==col[12] and blc["_atom_site.auth_comp_id"][i]==col[14]: 
                    binder_element_collector.append(type_symbol) 
            if set(binder_element_collector).difference(Acceptable_Element): # We screen the binder when it is in the adduct. If it has metal element, we skip the bond record.
                if debug: print(f'The binder in this record is invalid because it contains {set(binder_element_collector).difference(Acceptable_Element)}')
                continue    
            cbr.append(','.join(col))
    return cbr

import os, time, datetime, multiprocessing   
def mine():
    with multiprocessing.Pool(multiprocessing.cpu_count()) as p, open('progress', 'w') as fw1:  
        jobs = {}
        for subdir in os.scandir(rcsb_cif_dir): 
            if subdir.is_dir():
                for cif in os.scandir(subdir.path):
                    if cif.is_file():
                        job = p.apply_async(get_covalent_bond_record, (cif.name[:4].upper(),))
                        jobs.update({cif.name: job})  
             
        num_jobs = len(jobs)  
        print(f'{multiprocessing.cpu_count()} processors are working on {num_jobs} jobs @ {datetime.datetime.now().isoformat()}', file=fw1)
        fw1.flush()
        
        timestamp=time.time()
        wait_time=600
        for name, job in jobs.items(): 
            try:
                job.wait(timeout=600)
                if time.time() - timestamp > 60:
                    num_ready = sum([job.ready() for job in jobs.values()])
                    print(f'Finish rate: {num_ready} / {num_jobs} @ {datetime.datetime.now().isoformat()}', file=fw1)
                    fw1.flush()
                    timestamp=time.time()
            except TimeoutError:
                print(f"[{name}] Cannot mine it in {wait_time} seconds and got a multiprocessing.TimeoutError") 
        
        result = [job.get() for job in jobs.values() if job.ready()]  
        with open(pdb_cov_csv, 'w') as fw2:  
            [print(j, file=fw2) for i in result for j in i if i]
            
        print(f'Finish rate: {num_ready} / {num_jobs} @ {datetime.datetime.now().isoformat()}', file=fw1)        
        print(f'Finish mining @ {datetime.datetime.now().isoformat()}', file=fw1) 
        print(f'Problem with {", ".join([name for name, job in jobs.items() if not job.ready()] )}', file=fw1) 
#mine()

In [5]:
# Construct the dataframe of covalent bond records
import pandas as pd 
#pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

with open(pdb_cov_csv, 'r') as fr: 
    covalent_bond_records = [i for i in fr.read().splitlines()]
    covalent_bond_records.append("5F19,B,B,SER,SER,499,530,?,OG,?,polymer,B,B,OAS,OAS,0,0,?,C,?,non-polymer")
    df = pd.DataFrame(columns=['adduct_id', 'adduct_smiles', 'binder_atom_name','binder_id_in_adduct', 'binder_mol',  'binder_smiles', 'binder_type', 'chain_id', 'chain_name', 'common_name', 'covalent_bond_record', 'doi', 'note', 'pdb_id', 'pdb_title', 'reaction_type', 'recovery_strategy', 'res_atom_name', 'res_name', 'res_num', 'unp_accessionid', 'unp_resnum', 'warhead_name', 'warhead_smarts', 'multistep'])
    df = pd.concat([df,pd.DataFrame({'covalent_bond_record':covalent_bond_records})])  
    df['pdb_id'         ] = df.apply(lambda x: x.covalent_bond_record.split(',')[ 0], axis=1)
    df['adduct_id'      ] = df.apply(lambda x: '_'.join([x.covalent_bond_record.split(',')[i] for i in [0,2,4,6,7,8,9,12,14,16,17,18,19]]), axis=1)
    df['short_adduct_id'] = df.apply(lambda x: '_'.join([x.covalent_bond_record.split(',')[i] for i in [4,8,14,18]]), axis=1)
    df['multistep'      ] = df.apply(lambda x: math.nan, axis=1) 
    print(len(df), df['pdb_id'].nunique())
    df = df.set_index('adduct_id')
    df = df.drop(index=['7ARF_A_CYS_156_?_SG_?_A_RVW_401_?_S1_?', # 'The inhibitor contains aurum/gold element' although we cannot find the metal in component Aurothioglucose. 
                        "1EWP_A_CYS_25_?_SG_?_A_0I5_280_?_CM_?",  # "inaccessable work"
                        "1DNC_A_CYS_58_?_SG_?_A_GSH_1029_?_SG2_?"])  # "The inhibitor contains Ferrum/Iron element"

To_be_published = ['6YGV',"6WNP","7PCX","7PCW", "4EF8", "4EF9", "7JUJ", "7KG5", "7KLD", "7REP", "7SI9", "7TCZ", "7P1E", "7JP0", "7P1R", "7DKA", "7DHJ", "7L5T", "7BDR", "7D57", "7DQ0", "7DQ8", "7EMY", "7EN1", "7EN2", "7T3Y", "7T3Z", "7TE0", "7BAG", "7BAP", "7R84", "7R85", "7R86", "7JZO", "7JZQ", "7JZR", "7JXW", "7JXI", "7JXH", "7JXL", "7JXP", "7JXK", "7K1H", "7T4A", "7T42", "7T46", "7T41", "7T45", "7T40", "7T49", "7T44", "7T48", "7T4B", "7T43", "7DGI", "7DGH", "7DGG", "7DGB", "7DGF", "7LLD", "6VEZ", "7AJO", "7AJX", "7LXT", "7LXU", "7LXV", "7B12", "7B97", "7B9A", "7B9I","7KYU","4YD4","1ZWN","1ZUR","5EA9","1AMZ","1ASD","1ASE","1AU8","1BRU","1E51","1E5E","1EWL","1EWM","1EWO","1F2E","1FDJ","1H5X","1HXC","1J9C","1JCN","1K6R","1K6S","1NFB","1NIQ","1PVJ","1RWV","1TDK","1TH4","1UYQ","1V6C","1WVM","1Y9Z","1ZL0","1ZPZ","1ZSL","1ZTJ","1ZTK","1ZTL","2B0C","2B5Z","2B8G","2B8O","2C20","2DC6","2DC7","2DC8","2DC9","2DCA","2DCB","2DCC","2DCD","2FQ9","2FRA","2FRQ","2FT2","2FYE","2G6D","2GDD","2GYQ","2HJH","2IBI","2IDF","2IPP","2M5P","2M5Q","2M5R","2MVU","2MVV","2MY5","2O30","2OZU","2P0W","2P86","2PB1","2QSI","2TGD","2XD5","2XF8","2YAN","2YMW","2YNB","2ZQ9","2ZQA","2ZQC","2ZQD","2ZWL","3ABW","3B0R","3B75","3B9H","3BFC","3BFF","3BFG","3BHL","3BHR","3CIU","3CR6","3D1L","3D6F","3D6H","3D6M","3F9D","3FA6","3FJW","3FSL","3FZ8","3HZ7","3IHP","3IOQ","3IR4","3K2G","3KZH","3LY4","3M2Z","3M5T","3MUO","3MWA","3N6I","3N7W","3N8L","3N8R","3NBL","3NC8","3NCK","3NDE","3NDG","3P4U","3PJJ","3QGJ","3QRH","3QVO","3S19","3S3J","3S3P","3S3S","3SGU","3SW6","3SZN","3TAK","3TDF","3TIT","3TIU","3TNS","3TNT","3TPN","3UXJ","3VPL","3W6H","3W6I","3ZNT","3ZZ5","3ZZ6","3ZZ7","3ZZ8","3ZZ9","3ZZA","3ZZB","3ZZC","3ZZD","4A5R","4APD","4B4X","4B4Z","4BEN","4BPV","4BQV","4BS6","4CAZ","4CBB","4DZT","4E6W","4E6X","4EBL","4EBN","4EBP","4H3D","4H46","4IZT","4IZU","4IZV","4IZW","4JXG","4KEN","4KG2","4KG5","4KKY","4KS6","4L1O","4N1X","4OE7","4ONV","4OUK","4P8H","4PFA","4PO3","4Q5F","4Q5Q","4Q5R","4QDE","4QDV","4QEB","4QJG","4QR7","4QTF","4R0Q","4R1G","4R23","4R3J","4R7U","4RA7","4U4M","4UF4","4V0G","4V37","4WAV","4X3O","4X3P","4ZAR","5AQE","5ARB","5ARC","5ARD","5BPK","5C55","5C5N","5C5O","5DSH","5DV3","5DV6","5DV8","5DVC","5E93","5EHX","5EI5","5F2G","5F4S","5F58","5F6B","5F7G","5FAZ","5FEN","5FF7","5FFH","5FJG","5G0M","5GIY","5HY8","5HZE","5IEL","5J6B","5L20","5M4Z","5MHL","5MHM","5MHN","5MHO","5MMV","5NTJ","5OUC","5QH8","5QH9","5QHA","5QHH","5RGT","5RH5","5RH6","5RH7","5RH9","5RHA","5RL0","5RL1","5RL2","5RL3","5RL4","5RL5","5RXV","5TCK","5U4H","5U6G","5UI3","5VQH","5W1Y","5WI5","5WPX","5WPY","5WXS","5WYI","5XLU","5XXD","5XXG","5XXJ","5Y9B","5YD6","5YXZ","5YY1","5ZPU","6AB9","6ABA","6AZI","6BOI","6BPF","6BV5","6BV6","6BV8","6CN1","6DBB","6DE6","6EFH","6EMB","6ERS","6FEF","6FFW","6FOR","6FQE","6FU0","6FU6","6FVK","6FVP","6FW1","6FWE","6G0M","6G1J","6G23","6G3L","6G3M","6G88","6GT8","6GV2","6H7S","6HR6","6HR9","6I2H","6JI6","6KF7","6KVS","6KW0","6L65","6L66","6L71","6LIG","6M73","6M74","6MGU","6MK9","6N7L","6NKJ","6NNX","6NNY","6NOE","6NTG","6NTL","6NTM","6NTN","6NTO","6NWE","6OVI","6OWC","6P68","6P69","6PEL","6PGS","6PZP","6QIH","6QL8","6QLM","6QLW","6QLX","6QUD","6QVM","6R26","6R27","6RFK","6S63","6SYN","6T35","6T8A","6TCX","6THO","6TI1","6TI4","6TIX","6TSE","6TUH","6TVN","6UMC","6UX6","6V8C","6VDT","6W2Z","6W4N","6WHF","6WIF","6WSB","6X8H","6X8I","6X8J","6X8K","6X8L","6XFS","6XJ3","6XL4","6YR3","6YRH","6YRM","6YRT","6YS0","6YX8","6YYF","6YYG","6YYK","6Z2E","6Z7H","6Z7J","6Z7K","6ZRE","6ZRH","6ZRI","6ZRJ","6ZRP","6ZU8","6ZVU","6ZVV","6ZVW","6ZVX","6ZVY","6ZXI","7A3G","7A3I","7A3J","7A3L","7AYQ","7AYR","7BPH","7BWD","7C8B","7C8L","7C8R","7C8T","7CMP","7CX9","7DZD","7FAY","7FAZ","7JR4","7JYC","7K40","7K6D","7K6E","7LBN","7LHJ","7LHM","7LHN","7LHO","7LZT","7LZU","7LZV","7LZW","7LZX","7LZY","7LZZ","7M00","7M01","7M02","7M03","7M04","7MJF","7MNG","7MRR","7NBR","7NBS","7NBY","7OR4","7OZ7","7RN0"]
df = df.drop(index=df[df['pdb_id'].isin(To_be_published)].index)
print(len(df), df['pdb_id'].nunique(), df['short_adduct_id'].nunique())

with open('data/faulty_adduct.txt', 'r') as fr: 
    index_faulty_adduct = [i.strip() for i in fr.readlines()]
    df = df.drop(index=index_faulty_adduct, errors = 'ignore')
    
_df = pd.read_csv('data/knowledge_base.csv')
_df = _df.set_index('adduct_id')
df.update(_df) 


df['chain_id'           ] = df.apply(lambda x: x.covalent_bond_record.split(',')[ 2], axis=1)
df['res_name'           ] = df.apply(lambda x: x.covalent_bond_record.split(',')[ 4], axis=1)
df['res_num'            ] = df.apply(lambda x: x.covalent_bond_record.split(',')[ 6], axis=1)
df['res_atom_name'      ] = df.apply(lambda x: x.covalent_bond_record.split(',')[ 8], axis=1)
df['binder_id_in_adduct'] = df.apply(lambda x: x.covalent_bond_record.split(',')[14], axis=1)
df['binder_atom_name'   ] = df.apply(lambda x: x.covalent_bond_record.split(',')[18], axis=1)
print(len(df), df['pdb_id'].nunique())

13565 6301
12468 5754 3194
11951 5500


In [6]:
# Get doi for each PDB id
import re
re_date = re.compile('([12]\d{3}-(0[1-9]|1[0-2])-(0[1-9]|[12]\d|3[01]))') 

import json, time
with open('data/doi.json', 'r') as fr: doi = json.load(fr)  # This doi file is directly mined from mmCIF in advance. But some mmCIF lacks a DOI.
# We have to fetch the DOI from echa PDB's RCSB webpage like http://www.rcsb.org/structure/1PWC
def get_doi_by_pdb_id(pdb_id): 
    if pdb_id in doi: return None     
    time.sleep(1)
    soup = bs4.BeautifulSoup(urllib.request.urlopen(f'http://www.rcsb.org/structure/{pdb_id.lower()}').read(), 'lxml') 
    primarycitation = soup.find_all(id="primarycitation") 
    if primarycitation[0].text[-16:] == 'To be published.':  
        
        author = primarycitation[0].find('a').text  
        try:
            article_title = primarycitation[0].h4.text 
        except:
            article_title = "no_title"
        release_date = re.search(re_date, soup.find(id='header_deposited-released-dates').text).group()
        published_n_days_ago = datetime.datetime.now() - datetime.datetime.strptime(release_date, '%Y-%m-%d')
        if published_n_days_ago.days > 365 * 2:
            print('{"%s": "Release structure 2 years ago but published no article",author":"%s","release_date","%s",article_title": """%s"""},' % (pdb_id, author, release_date,article_title))
            return 
        else: 
            print('{"%s": "No DOI is available on RCSB website because the article is to be published.","author":"%s","release_date","%s","article_title": "%s"},' % (pdb_id, author, release_date,article_title))
            return 
    primarycitation = primarycitation[0].find_all(id="pubmedDOI")
    if len(primarycitation) == 1:  
        print('{"%s": "%s"},' % (pdb_id, primarycitation[0].text[9:]))
        return 
    elif len(primarycitation) == 0:
        print('{"%s": "No DOI is available on RCSB website for no reason."},' % (pdb_id))
        return 
    else:
        raise PDB_Cov_Exception(doi)
_ = [get_doi_by_pdb_id(i) for i in df['pdb_id'].unique()] 
df['doi'] = df.apply(lambda x: doi[x.pdb_id], axis=1) 

In [7]:
# Parse and reconstruct adduct from Definition File and save the adduct to a mol file
def get_adduct_pdb_from_covalent_bond_record(cbr, debug=False, test=False):
    col = cbr.split(',')
    pdb_id,res_name,res_atom_name,binder_name,binder_atom_name=[col[i] for i in [0,4,8,14,18]]
    try:
        pdb_id = pdb_id.lower()
        blc = Bio.PDB.MMCIF2Dict.MMCIF2Dict(f'{rcsb_cif_dir}/{pdb_id[1:3]}/{pdb_id}.cif')
        if debug: print(f'We have local copy of {pdb_id}.cif')
    except FileNotFoundError: 
        r = urllib.request.urlopen(f'http://files.rcsb.org/download/{pdb_id}.cif')
        f = io.StringIO(r.read().decode())
        if debug: print(f'We fetched {pdb_id}.cif from RCSB PDB')
        blc = Bio.PDB.MMCIF2Dict.MMCIF2Dict(f)
    atom_name_in_adduct = []
    for i, label_entity_id in enumerate(blc['_atom_site.label_entity_id']):
        if  blc["_atom_site.label_asym_id"    ][i] == col[11] and \
            blc["_atom_site.auth_asym_id"     ][i] == col[12] and \
            blc["_atom_site.label_comp_id"    ][i] == col[13] and \
            blc["_atom_site.auth_comp_id"     ][i] == col[14] and \
            blc["_atom_site.label_seq_id"     ][i] == col[15] and \
            blc["_atom_site.auth_seq_id"      ][i] == col[16] and \
            blc["_atom_site.pdbx_PDB_ins_code"][i] == col[17]: 
            atom_name_in_adduct.append(blc["_atom_site.auth_atom_id"][i]) 

    try: 
        blc = Bio.PDB.MMCIF2Dict.MMCIF2Dict(f'{cpd_cif}/{binder_name}.cif')
        if debug: print(f'We have local copy of {binder_name}.cif')
    except FileNotFoundError:
        r = urllib.request.urlopen(f'http://files.rcsb.org/ligands/view/{binder_name}.cif')
        r = r.read()
        with open(f'{cpd_cif}/{binder_name}.cif', 'wb') as fw: fw.write(r)
        f = io.StringIO(r.decode())
        if debug: print(f'We fetched {binder_name}.cif from RCSB PDB')
        blc = Bio.PDB.MMCIF2Dict.MMCIF2Dict(f)
      
    Yes_to_True = {'Y':True, 'N':False}
    symbol_map  = {'H': 'H', 'HE': 'He', 'LI': 'Li', 'BE': 'Be', 'B': 'B', 'C': 'C', 'N': 'N', 'O': 'O', 'F': 'F', 'NE': 'Ne', 'NA': 'Na', 'MG': 'Mg', 'AL': 'Al', 'SI': 'Si', 'P': 'P', 'S': 'S', 'CL': 'Cl', 'AR': 'Ar', 'K': 'K', 'CA': 'Ca', 'SC': 'Sc', 'TI': 'Ti', 'V': 'V', 'CR': 'Cr', 'MN': 'Mn', 'FE': 'Fe', 'CO': 'Co', 'NI': 'Ni', 'CU': 'Cu', 'ZN': 'Zn', 'GA': 'Ga', 'GE': 'Ge', 'AS': 'As', 'SE': 'Se', 'BR': 'Br', 'KR': 'Kr', 'RB': 'Rb', 'SR': 'Sr', 'Y': 'Y', 'ZR': 'Zr', 'NB': 'Nb', 'MO': 'Mo', 'TC': 'Tc', 'RU': 'Ru', 'RH': 'Rh', 'PD': 'Pd', 'AG': 'Ag', 'CD': 'Cd', 'IN': 'In', 'SN': 'Sn', 'SB': 'Sb', 'TE': 'Te', 'I': 'I', 'XE': 'Xe', 'CS': 'Cs', 'BA': 'Ba', 'LA': 'La', 'HF': 'Hf', 'TA': 'Ta', 'W': 'W', 'RE': 'Re', 'OS': 'Os', 'IR': 'Ir', 'PT': 'Pt', 'AU': 'Au', 'HG': 'Hg', 'TL': 'Tl', 'PB': 'Pb', 'BI': 'Bi', 'PO': 'Po', 'AT': 'At', 'RN': 'Rn', 'FR': 'Fr', 'RA': 'Ra', 'AC': 'Ac', 'RF': 'Rf', 'DB': 'Db', 'SG': 'Sg', 'BH': 'Bh', 'HS': 'Hs', 'MT': 'Mt', 'DS': 'Ds', 'RG': 'Rg', 'CN': 'Cn', 'NH': 'Nh', 'FL': 'Fl', 'MC': 'Mc', 'LV': 'Lv', 'TS': 'Ts', 'OG': 'Og', 'CE': 'Ce', 'PR': 'Pr', 'ND': 'Nd', 'PM': 'Pm', 'SM': 'Sm', 'EU': 'Eu', 'GD': 'Gd', 'TB': 'Tb', 'DY': 'Dy', 'HO': 'Ho', 'ER': 'Er', 'TM': 'Tm', 'YB': 'Yb', 'LU': 'Lu', 'TH': 'Th', 'PA': 'Pa', 'U': 'U', 'NP': 'Np', 'PU': 'Pu', 'AM': 'Am', 'CM': 'Cm', 'BK': 'Bk', 'CF': 'Cf', 'ES': 'Es', 'FM': 'Fm', 'MD': 'Md', 'NO': 'No', 'LR': 'Lr'}
    bond_types  = {'SING':Chem.BondType.SINGLE, 'DOUB':Chem.BondType.DOUBLE, 'TRIP':Chem.BondType.TRIPLE}
    
    edMol       = Chem.EditableMol(Chem.Mol()) 
    # Atom Related Section 
    RESNs    = blc['_chem_comp_atom.comp_id'                  ]   
    ATMNs    = blc['_chem_comp_atom.atom_id'                  ]  
    SYMBOLs  = blc['_chem_comp_atom.type_symbol'              ]      
    CHARGEs  = blc['_chem_comp_atom.charge'                   ]   
    AROMFs   = blc['_chem_comp_atom.pdbx_aromatic_flag'       ]
    Xs       = blc['_chem_comp_atom.model_Cartn_x'            ] 
    Ys       = blc['_chem_comp_atom.model_Cartn_y'            ] 
    Zs       = blc['_chem_comp_atom.model_Cartn_z'            ]  
    Xs_ideal = blc['_chem_comp_atom.pdbx_model_Cartn_x_ideal' ]
    Ys_ideal = blc['_chem_comp_atom.pdbx_model_Cartn_y_ideal' ]  
    Zs_ideal = blc['_chem_comp_atom.pdbx_model_Cartn_z_ideal' ] 
    idx_map     = {}
    for resn, atom_name, symbol, charge, aromatic in zip(RESNs, ATMNs, SYMBOLs, CHARGEs, AROMFs): 
        atom = Chem.Atom(symbol_map[symbol]) 
        atom.SetMonomerInfo(Chem.AtomPDBResidueInfo(atomName = atom_name, residueName = resn)) 
        if charge!='?': atom.SetFormalCharge(int(charge))
        atom.SetIsAromatic(Yes_to_True[aromatic])
        atom_id = edMol.AddAtom(atom) 
        idx_map[atom_name] = atom_id  
        
    # Bond Related Section 
    BGN_ATMNs   = blc['_chem_comp_bond.atom_id_1'   ]   
    END_ATMNs   = blc['_chem_comp_bond.atom_id_2'   ] 
    ORDERs      = blc['_chem_comp_bond.value_order' ]    
    for begin_atom_name, end_atom_name, bond_type in zip(BGN_ATMNs, END_ATMNs, ORDERs):
        begin_atom_idx = idx_map[begin_atom_name] 
        end_atom_idx   = idx_map[end_atom_name  ]
        bond_type      = bond_types[bond_type]
        edMol.AddBond(begin_atom_idx, end_atom_idx, bond_type)  
    
    mol = edMol.GetMol()  
    try:
        Chem.SanitizeMol(mol)
    except: 
        raise PDB_Cov_Exception(f'Sanitation Failure for http://www.rcsb.org/ligand/{binder_name} in {pdb_id}')
    
    # Conformation
    if not '?' in Xs:
        pass # Xs, Ys, Zs = Xs, Ys, Zs
    elif '?' not in Xs_ideal: 
        Xs, Ys, Zs = Xs_ideal, Ys_ideal, Zs_ideal
    else: 
        raise PDB_Cov_Exception(f'Problem with atomic coordinate for http://www.rcsb.org/ligand/{binder_name} in {pdb_id}')
    conf = Chem.Conformer() 
    for atom_id, (x, y, z) in enumerate(zip(Xs, Ys, Zs)):
        conf.SetAtomPosition(atom_id, Point3D(float(x), float(y), float(z)))   
    mol.AddConformer(conf, assignId=True)
    AllChem.AssignAtomChiralTagsFromStructure(mol)
    mol.RemoveAllConformers() 
    
    try:
        mol = Chem.RemoveHs(mol)
    except:
        raise PDB_Cov_Exception(f'RemoveHs Failure for http://www.rcsb.org/ligand/{binder_name} in {pdb_id}') 
        
    AN = lambda atom: atom.GetMonomerInfo().GetName().strip() # Atom Name   
    # We begin with the ligand-side-covalent-bond-end-atom on ligand 
    BinderEndAtom_template = [atom for atom in mol.GetAtoms() if AN(atom) == binder_atom_name][0]          # Atom In tPDB
    atom_name_around_ligand_end_atom_in_template = set([AN(atom) for atom in BinderEndAtom_template.GetNeighbors()])
    more_atom_in_template = set(atom_name_around_ligand_end_atom_in_template).difference(set(atom_name_in_adduct))
      
    amino_acid_atom_info = Chem.AtomPDBResidueInfo(residueName=res_name, atomName=res_atom_name)
    atom_number = {'O':52, 'N':83, 'S':84}[res_atom_name[0]] # O: Te, N: Bi, S:Po
    if len(more_atom_in_template)==1: # Substitution
        if debug: print('substitution')
        more_atom_in_template = more_atom_in_template.pop()
        assumedAminoAcidEndAtom = [atom for atom in mol.GetAtoms() if AN(atom) == more_atom_in_template][0] # Atom In tPDB  
        assumedAminoAcidEndAtom.SetAtomicNum(atom_number) 
        assumedAminoAcidEndAtom.SetMonomerInfo(amino_acid_atom_info) 
        sio = sys.stderr = io.StringIO()
        try:  
            atom_idxs = [atom.GetIdx() for atom in assumedAminoAcidEndAtom.GetNeighbors() if AN(atom)!=binder_atom_name] 
            if atom_idxs:
                edMol = Chem.EditableMol(mol)
                [edMol.RemoveBond(i, assumedAminoAcidEndAtom.GetIdx()) for i in sorted(atom_idxs, reverse=True)]
                mol = edMol.GetMol()
                AtIdx = mol.GetSubstructMatch(Chem.MolFromSmarts(f'[#{atom_number}]'))[0] 
                atom_idxs_to_remove = [atom_idxs for atom_idxs in Chem.GetMolFrags(mol) if AtIdx not in atom_idxs]
                atom_idxs_to_remove = sorted([j for i in atom_idxs_to_remove for j in i], reverse=True) 
                edMol = Chem.EditableMol(mol) 
                [edMol.RemoveAtom(i) for i in atom_idxs_to_remove]
                mol = edMol.GetMol()  
                AtIdx = mol.GetSubstructMatch(Chem.MolFromSmarts(f'[#{atom_number}]'))[0]
                assumedAminoAcidEndAtom = mol.GetAtomWithIdx(AtIdx)
                if assumedAminoAcidEndAtom.GetIsAromatic(): assumedAminoAcidEndAtom.SetIsAromatic(False) # ValueError: Sanitization error: non-ring atom 14 marked aromatic
                Chem.SanitizeMol(mol)  
            if test: mol=b'substitution'
        except Exception as e:  
            raise PDB_Cov_Exception(f'Unable to recover adduct for http://www.rcsb.org/ligand/{binder_name} in {pdb_id} in Substitution')  
    elif len(more_atom_in_template)==0: # Reaction mechanism tend to be addition
        if debug: print('addition')
        ed = Chem.EditableMol(mol)
        atom = Chem.Atom(atom_number)
        atom.SetMonomerInfo(amino_acid_atom_info) 
        aaEndAtomIdx = ed.AddAtom(atom) 
        ed.AddBond(BinderEndAtom_template.GetIdx(), aaEndAtomIdx, order=Chem.rdchem.BondType.SINGLE)
        mol = ed.GetMol() 
        if mol.GetSubstructMatch(Chem.MolFromSmarts('[#52,#83,#84]~[a;X3]')): 
            Chem.SanitizeMol(mol)
        elif mol.GetSubstructMatch(Chem.MolFromSmarts('[#52,#83,#84]~[a;X4]')): 
            mol = Chem.MolFromSmarts(Chem.MolToSmarts(mol))
            Chem.SanitizeMol(mol)
            raise PDB_Cov_Exception(f'Explicit valence for atom')
        try:
            sio = sys.stderr = io.StringIO() 
            Chem.Draw.MolToImage(mol)
            mol.RemoveAllConformers()
            if test: mol=b'addition'
        except Exception as e:
            raise PDB_Cov_Exception(f'Unable to recover adduct for http://www.rcsb.org/ligand/{binder_name} in {pdb_id} in Addition method ')
    else:
        raise PDB_Cov_Exception(f'Unable to recover adduct for http://www.rcsb.org/ligand/{binder_name} in {pdb_id} because of ???')
    
    
    try:
        mol = Chem.RemoveAllHs(mol)  
    except:
        pass 
    return mol

import os
import time
import datetime
 
date = datetime.datetime.now()
modTime = time.mktime(date.timetuple())

def generate_adduct_bin(cbr, test=False):
    col = cbr.split(',')
    _   = '_'.join([col[i] for i in [4,8,14,18]]) 
    __   = f'{adduct_bin}/{_}'
    if test: __   = f'{adduct_bin}/{_}.test'
    if os.path.isfile(__): 
        os.utime(__, (modTime, modTime))
        return None 
    try:
        mol = get_adduct_pdb_from_covalent_bond_record(cbr, test=test) 
        with open(__, 'wb') as fw: 
            if test:
                fw.write(mol)  
            else:
                fw.write(mol.ToBinary())  
    except Exception as e:
        print(_, '\t\t', e, '\t\t\t', cbr)  

for cbr in df['covalent_bond_record']:
    generate_adduct_bin(cbr, test=False)
# find . -amin +600 -exec ls -l {} \;
# executed in 1h 56m 23s, finished 04:36:02 2021-05-21

In [8]:
ad_id = lambda x: '_'.join([x.split(',')[i] for i in [0,2,4,6,7,8,9,12,14,16,17,18,19]]) 
#ad_id('7VFV,A,A,TYR,TYR,1289,1289,?,OH,?,polymer,AA,A,6IX,6IX,.,2421,?,C32,?,non-polymer')

In [9]:
# read mol file and put the mol in Dataframe
import math
def get_adduct_pdb(short_adduct_id, adduct_smiles): 
    if isinstance(adduct_smiles,str):
        m = Chem.MolFromSmiles(adduct_smiles)
        if m:
            return m
        else:
            raise
    elif os.path.isfile(f'{adduct_bin}/{short_adduct_id}'):
        with open(f'{adduct_bin}/{short_adduct_id}', 'rb') as fr: 
            return Chem.Mol(fr.read())
    else:
        return math.nan 
    #chemicalIsomeric = b.find(attrs={'id':"chemicalIsomeric"}).td.text 
df['adduct_pdb'] = df.apply(lambda x: get_adduct_pdb(x.short_adduct_id, x.adduct_smiles), axis=1)

In [10]:
# This is a deprecated binder recovery strategy, which assumes the recovered binder is same if the adduct is same.
_ = df[df['binder_type'].notna()]
_ = _.drop_duplicates(subset=['short_adduct_id'])
_ = _[['binder_smiles','binder_type','reaction_type','recovery_strategy','warhead_name','short_adduct_id']] 
lig_match={}
for i in _.to_dict('records'): 
    short_adduct_id = i.pop('short_adduct_id') 
    lig_match.update({short_adduct_id:i})

for i, row in df[df['binder_type'].isna()].iterrows(): 
    short_adduct_id = row['short_adduct_id']  
    if short_adduct_id in lig_match:
        row = row.to_dict()
        row.update(lig_match[short_adduct_id])
        row.update({'recovery_strategy':'ligand_matching'})
        df.update(pd.DataFrame(row, index=[i])) 

#df = df[~(df['binder_type'].isin(['non-covalent','cofactor','nucleic_acid_adduct','part_of_peptide', 'part_of_cyclopeptide','appendage']))]
df = df[~(df['binder_type'].isin(['non-covalent','cofactor', 'nucleic_acid_adduct','part_of_peptide', 'part_of_cyclopeptide', 'appendage']))]

In [11]:
# Convert mol to SMILES string and then convert SMILES string back to mol 
def MolFromSmiles(s,m):
    if not isinstance(m, float):
        return m
    if isinstance(s,str):
        m = Chem.MolFromSmiles(s)
        if not m: print(s)
        return m
    return s
df['binder_mol'] = df.apply(lambda x: MolFromSmiles(x.binder_smiles, x.binder_mol), axis=1)

In [12]:
# Use characteristic substructure substitution to recover the binder
class WARHEAD: 
    def smarts(self, mol, smarts, name=None, reaction_type=None, note=None):
        self.mol            = Chem.MolFromSmarts(smarts) 
        self.warhead_name   = name 
        self.reaction_type  = reaction_type
        self.note           = note
        return mol.HasSubstructMatch(self.mol) 
    
def get_binder_pdb_by_automatic_recovery(adduct_pdb, debug=False): 
    mol = Chem.Mol(adduct_pdb)
    warhead = WARHEAD() 
    print('##############')
    if   warhead.smarts(mol, "[#52,#83,#84]~[#5]"):
        Achor, B = mol.GetSubstructMatch(warhead.mol) 
        if   warhead.smarts(mol, '[#52,#83,#84]~[#5;R]~[#6,#7]' , name='Cyclic_Boronic_Acid', reaction_type='Addition'):
            if mol.HasSubstructMatch(Chem.MolFromSmarts('[#52,#83,#84]~[#5D4]')): 
                mol = AllChem.DeleteSubstructs(mol, Chem.MolFromSmarts("[#52,#83,#84]"))   
            elif mol.HasSubstructMatch(Chem.MolFromSmarts('[#52,#83,#84]~[#5D3]')):
                Achor = mol.GetSubstructMatch(Chem.MolFromSmarts("[#52,#83,#84]"))[0]
                mol.GetAtomWithIdx(Achor).SetAtomicNum(8) 
            else:
                raise PDB_Cov_Exception('[ Curate by you own~ ]')   
            Chem.GetSymmSSSR(mol)  
        elif warhead.smarts(mol, '[#52,#83,#84]~[#5;R0]~[#6,#7]', name='Boronic_Acid'       , reaction_type='Addition'): 
            if mol.HasSubstructMatch(Chem.MolFromSmarts('[#52,#83,#84]~[#5D4]')): 
                mol = AllChem.DeleteSubstructs(mol, Chem.MolFromSmarts("[#52,#83,#84]"))  
            elif mol.HasSubstructMatch(Chem.MolFromSmarts('[#52,#83,#84]~[#5D3]')):
                Achor = mol.GetSubstructMatch(Chem.MolFromSmarts("[#52,#83,#84]"))[0]
                mol.GetAtomWithIdx(Achor).SetAtomicNum(8) 
            else:
                raise PDB_Cov_Exception('[ Curate by you own~ ]')   
        B, = mol.GetSubstructMatch(Chem.MolFromSmarts('[#5]')) 
        B = mol.GetAtomWithIdx(B)
        if B.GetDegree() == 3: B.SetFormalCharge(0)
        B.UpdatePropertyCache() 
            
    elif warhead.smarts(mol, '[#52,#83,#84]C(=O)CCN=[C;R][C;R]', 'β-lactam', reaction_type='Ring-opening'):
        Achor, C1, _, _, _, N, C2, C3 = mol.GetSubstructMatch(warhead.mol) 
        mol.GetBondBetweenAtoms(N, C2).SetBondType(Chem.rdchem.BondType.SINGLE)  
        mol.GetBondBetweenAtoms(C2, C3).SetBondType(Chem.rdchem.BondType.DOUBLE)
        mol.GetAtomWithIdx(C3).SetNumExplicitHs(0)
        edMol = Chem.EditableMol(mol)
        edMol.AddBond(C1, N, Chem.rdchem.BondType.SINGLE) 
        edMol.RemoveAtom(Achor)
        mol = edMol.GetMol()    
        
    elif warhead.smarts(mol, '[#52,#83,#84]C(=O)CCN', 'β-lactam', reaction_type='Ring-opening'):
        Achor, C, _, _, _, N = mol.GetSubstructMatch(warhead.mol) 
        edMol = Chem.EditableMol(mol)
        edMol.AddBond(C, N, Chem.rdchem.BondType.SINGLE) 
        edMol.RemoveAtom(Achor)
        mol = edMol.GetMol() 
            
    elif warhead.smarts(mol, '[#52,#83,#84]C(=O)CCO', name='β-lactone', reaction_type='Ring-opening'):
        Achor, C, _, _, _, O = mol.GetSubstructMatch(warhead.mol) 
        edMol = Chem.EditableMol(mol)
        edMol.AddBond(C, O, Chem.rdchem.BondType.SINGLE) 
        edMol.RemoveAtom(Achor)
        mol = edMol.GetMol()
    
    elif warhead.smarts(mol, '[#52,#83,#84]C(C=O)C(O)C=O', name='Epoxide', reaction_type='Ring-opening'): 
        Achor, C, _, _, _, O, _, _ = mol.GetSubstructMatch(warhead.mol) 
        edMol = Chem.EditableMol(mol)
        edMol.AddBond(C, O, Chem.rdchem.BondType.SINGLE) 
        edMol.RemoveAtom(Achor)
        mol = edMol.GetMol()  
            
    elif warhead.smarts(mol, '[#52,#83,#84]C~C[CD3]([#6])=O', name='Vinylketone', reaction_type='Addition'):    
        Achor, Cb, Ca, _, _, _ = mol.GetSubstructMatch(warhead.mol) 
        mol.GetBondBetweenAtoms(Ca, Cb).SetBondType(Chem.rdchem.BondType.SINGLE)
        mol = Chem.Mol(mol)
        mol.GetBondBetweenAtoms(Ca, Cb).SetBondType(Chem.rdchem.BondType.DOUBLE)
        mol.GetAtomWithIdx(Cb).SetNumExplicitHs(0)
        Chem.SanitizeMol(mol, sanitizeOps=Chem.SANITIZE_SETHYBRIDIZATION)
        Ca = mol.GetAtomWithIdx(Ca)
        Ca.SetNumExplicitHs(0)
        edMol = Chem.EditableMol(mol) 
        edMol.RemoveAtom(Achor)
        mol = edMol.GetMol()  
        
    elif warhead.smarts(mol, '[#52,#83,#84]C~C[CD3]([#7,#16])=O', name='Michael_Acceptor', reaction_type='Addition'):    
        Achor, Cb, Ca, _, _, _ = mol.GetSubstructMatch(warhead.mol) 
        mol.GetBondBetweenAtoms(Ca, Cb).SetBondType(Chem.rdchem.BondType.SINGLE)
        mol = Chem.Mol(mol)
        mol.GetBondBetweenAtoms(Ca, Cb).SetBondType(Chem.rdchem.BondType.DOUBLE)
        mol.GetAtomWithIdx(Cb).SetNumExplicitHs(0)
        Chem.SanitizeMol(mol, sanitizeOps=Chem.SANITIZE_SETHYBRIDIZATION)
        Ca = mol.GetAtomWithIdx(Ca)
        Ca.SetNumExplicitHs(0)
        edMol = Chem.EditableMol(mol) 
        edMol.RemoveAtom(Achor)
        mol = edMol.GetMol()  
            
    elif warhead.smarts(mol, '[#52,#83,#84]C~C[CD3]([#8D1])=O', name='Acrylic_Acid', reaction_type='Addition'):    
        Achor, Cb, Ca, _, _, _ = mol.GetSubstructMatch(warhead.mol) 
        mol.GetBondBetweenAtoms(Ca, Cb).SetBondType(Chem.rdchem.BondType.SINGLE)
        mol = Chem.Mol(mol)
        mol.GetBondBetweenAtoms(Ca, Cb).SetBondType(Chem.rdchem.BondType.DOUBLE)
        mol.GetAtomWithIdx(Cb).SetNumExplicitHs(0)
        Chem.SanitizeMol(mol, sanitizeOps=Chem.SANITIZE_SETHYBRIDIZATION)
        Ca = mol.GetAtomWithIdx(Ca)
        Ca.SetNumExplicitHs(0)
        edMol = Chem.EditableMol(mol) 
        edMol.RemoveAtom(Achor)
        mol = edMol.GetMol()  
        
    elif warhead.smarts(mol, '[#52,#83,#84]C~C[CD3]([#8D2])=O', name='Acrylic_Ester', reaction_type='Addition'):    
        Achor, Cb, Ca, _, _, _ = mol.GetSubstructMatch(warhead.mol) 
        mol.GetBondBetweenAtoms(Ca, Cb).SetBondType(Chem.rdchem.BondType.SINGLE)
        mol = Chem.Mol(mol)
        mol.GetBondBetweenAtoms(Ca, Cb).SetBondType(Chem.rdchem.BondType.DOUBLE)
        mol.GetAtomWithIdx(Cb).SetNumExplicitHs(0)
        Chem.SanitizeMol(mol, sanitizeOps=Chem.SANITIZE_SETHYBRIDIZATION)
        Ca = mol.GetAtomWithIdx(Ca)
        Ca.SetNumExplicitHs(0)
        edMol = Chem.EditableMol(mol) 
        edMol.RemoveAtom(Achor)
        mol = edMol.GetMol()  
        
    elif warhead.smarts(mol, '[#52,#83,#84]C~C[CD2]=O', name='Vinylaldehyde', reaction_type='Addition'):    
        Achor, Cb, Ca, _, _ = mol.GetSubstructMatch(warhead.mol) 
        mol.GetBondBetweenAtoms(Ca, Cb).SetBondType(Chem.rdchem.BondType.SINGLE)
        mol = Chem.Mol(mol)
        mol.GetBondBetweenAtoms(Ca, Cb).SetBondType(Chem.rdchem.BondType.DOUBLE)
        mol.GetAtomWithIdx(Cb).SetNumExplicitHs(0)
        Chem.SanitizeMol(mol, sanitizeOps=Chem.SANITIZE_SETHYBRIDIZATION)
        Ca = mol.GetAtomWithIdx(Ca)
        Ca.SetNumExplicitHs(0)
        edMol = Chem.EditableMol(mol) 
        edMol.RemoveAtom(Achor)
        mol = edMol.GetMol()  
        
    elif warhead.smarts(mol, '[#52,#83,#84]C~CS(=O)=O', name='Vinylsulfonyl', reaction_type='Addition'):    
        Achor, Cb, Ca, _, _, _ = mol.GetSubstructMatch(warhead.mol) 
        mol.GetBondBetweenAtoms(Ca, Cb).SetBondType(Chem.rdchem.BondType.SINGLE)
        mol = Chem.Mol(mol)
        mol.GetBondBetweenAtoms(Ca, Cb).SetBondType(Chem.rdchem.BondType.DOUBLE)
        mol.GetAtomWithIdx(Cb).SetNumExplicitHs(0)
        Chem.SanitizeMol(mol, sanitizeOps=Chem.SANITIZE_SETHYBRIDIZATION)
        Ca = mol.GetAtomWithIdx(Ca)
        Ca.SetNumExplicitHs(0)
        edMol = Chem.EditableMol(mol) 
        edMol.RemoveAtom(Achor)
        mol = edMol.GetMol()  
                
    elif warhead.smarts(mol, '[#52,#83,#84]~[CD3](~[ND1])', name='Nitrile', reaction_type='Addition'):  
        Achor, C, N = mol.GetSubstructMatch(warhead.mol)
        mol.GetBondBetweenAtoms(C, N).SetBondType(Chem.rdchem.BondType.DOUBLE)  
        mol.GetBondBetweenAtoms(C, Achor).SetBondType(Chem.rdchem.BondType.SINGLE)  
        mol.GetAtomWithIdx(C).SetNumExplicitHs(0)
        mol = Chem.Mol(mol) 
        mol.GetBondBetweenAtoms(C, N).SetBondType(Chem.rdchem.BondType.TRIPLE)  
        edMol = Chem.EditableMol(mol)
        edMol.RemoveAtom(Achor)
        mol = edMol.GetMol() 
        Chem.SanitizeMol(mol, sanitizeOps=Chem.SANITIZE_SETHYBRIDIZATION)
        mol = Chem.MolFromSmiles(Chem.MolToSmiles(mol)) 
    
    elif warhead.smarts(mol, '[#52,#83,#84][#6](~[OD1])([#6])[#6]', name='Ketone', reaction_type='Addition'):   
        Achor, C, O, _, _ = mol.GetSubstructMatch(warhead.mol)
        mol.GetBondBetweenAtoms(C, O).SetBondType(Chem.rdchem.BondType.SINGLE) 
        mol.GetAtomWithIdx(C).SetNumExplicitHs(0)   
        mol.GetBondBetweenAtoms(C, O).SetBondType(Chem.rdchem.BondType.DOUBLE)
        edMol = Chem.EditableMol(mol)
        edMol.RemoveAtom(Achor)
        mol = edMol.GetMol() 
        Chem.SanitizeMol(mol, sanitizeOps=Chem.SANITIZE_SETHYBRIDIZATION) 
    
    elif warhead.smarts(mol, '[#52,#83,#84][#6](~[OD1])NCCNOS', name='γ-lactam', reaction_type='Ring-opening', note='Diazabicyclooctane'):  
        Achor, C, _, _, _, _, N, _, _ = mol.GetSubstructMatch(warhead.mol)
        edMol = Chem.EditableMol(mol)
        edMol.AddBond(C, N, Chem.rdchem.BondType.SINGLE)
        edMol.RemoveAtom(Achor)
        mol = edMol.GetMol() 
        Chem.SanitizeMol(mol, sanitizeOps=Chem.SANITIZE_SETHYBRIDIZATION) 
        
    elif warhead.smarts(mol, '[#52,#83,#84]=[#6;D3]', name='Carbonyl_Oxygen', reaction_type='Addition-Elimination', note='Schiff Base'):   
        Achor, = mol.GetSubstructMatch(Chem.MolFromSmarts('[#52,#83,#84]'))
        mol.GetAtomWithIdx(Achor).SetFormalCharge(1) 
        mol = Chem.Mol(mol)
        a = mol.GetAtomWithIdx(Achor)
        a.SetAtomicNum(8) 
        a.SetNumExplicitHs(0)
        a.SetFormalCharge(0) 
    
    elif warhead.smarts(mol, '[#52,#83,#84]~[#6;D3](~[#16])~[#7]', name='Isothiocyanate', reaction_type='Addition', note='Schiff Base'):    
        Achor, C, S, N = mol.GetSubstructMatch(warhead.mol) 
        mol.GetBondBetweenAtoms(C, N).SetBondType(Chem.rdchem.BondType.DOUBLE) 
        mol.GetBondBetweenAtoms(C, S).SetBondType(Chem.rdchem.BondType.SINGLE)  
        mol = Chem.Mol(mol)
        mol.GetBondBetweenAtoms(C, S).SetBondType(Chem.rdchem.BondType.DOUBLE)   
        edMol = Chem.EditableMol(mol)
        edMol.RemoveAtom(Achor)
        mol = edMol.GetMol() 
        Chem.SanitizeMol(mol, sanitizeOps=Chem.SANITIZE_SETHYBRIDIZATION) 
    
    else: 
        raise PDB_Cov_Exception('[ Curate by you own~ ]')  
    
    result = {i: warhead.__getattribute__(i) for i in ['warhead_name', 'reaction_type','note']}
    result.update({'binder_mol':mol}) 
    return result

In [13]:
# automatic: characteristic substructure substitution
for i, row in df[(df['binder_smiles'].isna()) & (df['binder_type'].isna())].iterrows():
    adduct_pdb = row['adduct_pdb']
    try:
        result = get_binder_pdb_by_automatic_recovery(adduct_pdb)  
        row = row.to_dict()
        row.update(result)
        row.update({'recovery_strategy':'automatic', 'binder_type':'inhibitor'})
        df.update(pd.DataFrame(row, index=[i])) 
    except Exception as e:
        print(e,'\t\t\t', i) 

In [14]:
# mine pdb's title and put it in the dataframe
collector = {}
with open('data/pdb_title_chain_name.json', 'r') as fr: collector = json.load(fr)
def get_title_and_name(PDBID):  
    if PDBID in collector: return collector[PDBID]
    try:
        pdbid = PDBID.lower()
        blc = Bio.PDB.MMCIF2Dict.MMCIF2Dict(f'{rcsb_cif_dir}/{pdbid[1:3]}/{pdbid}.cif') 
    except FileNotFoundError: 
        r = urllib.request.urlopen(f'http://files.rcsb.org/download/{pdbid}.cif')
        f = io.StringIO(r.read().decode()) 
        blc = Bio.PDB.MMCIF2Dict.MMCIF2Dict(f) 
        print('[web access] get_title_and_name:',pdbid)
    title = blc['_struct.title'][0] 
    type_={i:j for i,j,k in zip(blc['_entity.id'], blc['_entity.pdbx_description'],blc['_entity.type']) if k=='polymer'} 
    name = [(chain_id, type_[entity_id]) for entity_id, chain_ids in zip(blc['_entity_poly.entity_id'], blc['_entity_poly.pdbx_strand_id']) for chain_id in chain_ids.split(',') if entity_id in type_]
    return {PDBID:{'title':title,'name':dict(name)}} 

[collector.update(get_title_and_name(PDBID)) for PDBID in df['pdb_id'].unique()]
  
with open('data/pdb_title_chain_name.json', 'w') as fw: json.dump(collector, fw)

In [15]:
df['pdb_title' ] = df.apply(lambda x: collector[x.pdb_id]['title']             if x.pdb_id in collector else math.nan, axis=1)
df['chain_name'] = df.apply(lambda x: collector[x.pdb_id]['name' ][x.chain_id] if x.pdb_id in collector else math.nan, axis=1)

In [16]:
df[df['binder_mol'].isna()]['binder_type'].unique()

array(['substrate', 'linker', 'modifier', 'intermediate'], dtype=object)

In [17]:
q = df[df['binder_mol'].isna()]
q[q['binder_type'].isin([ 'inhibitor'])]  

,adduct_smiles,binder_atom_name,binder_id_in_adduct,binder_mol,binder_smiles,binder_type,chain_id,chain_name,common_name,covalent_bond_record,doi,note,pdb_id,pdb_title,reaction_type,recovery_strategy,res_atom_name,res_name,res_num,unp_accessionid,unp_resnum,warhead_name,warhead_smarts,multistep,short_adduct_id,adduct_pdb
adduct_id,,,,,,,,,,,,,,,,,,,,,,,,,,


In [18]:
# add warhead SMILES according to the name
import json
with open('data/warhead_smarts.json','r') as fr: warhead_smarts_from_name = json.load(fr)

def get_warhead_smarts(warhead_name, warhead_smarts):
    if isinstance(warhead_name, float) or isinstance(warhead_smarts, str): # No name
        return warhead_smarts
    else:
        if warhead_name not in warhead_smarts_from_name: 
            print(warhead_name)
            return None
        return warhead_smarts_from_name[warhead_name]
df['warhead_smarts'] = df.apply(lambda x: get_warhead_smarts(x.warhead_name, x.warhead_smarts), axis=1)

df[(df['warhead_name'].notna()) & (df['warhead_smarts'].isna())]['warhead_name'].unique()

array([], dtype=object)

In [19]:
from io import StringIO 
import sys 
Chem.WrapLogs()  

def MolToInchiKey(m,s):  
    sio = sys.stderr = StringIO()   
    if isinstance(m, float): 
        return math.nan 
    Chem.SanitizeMol(m)   
    ik = Chem.MolToInchiKey(m)
    e = sio.getvalue() 
    if e:
        print(s, e)
    return  ik 
df['InChIKey'] = df.apply(lambda x: MolToInchiKey(x.binder_mol,x.name), axis=1)  

In [20]:
import datetime # 2mins
print(len(df))
df.to_pickle(f'data/df_final.{datetime.datetime.now().isoformat()}.pkl', protocol=4)
df.to_pickle(f'data/df_latest.pkl', protocol=4)

8202
